In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("../data/new/monthly_dataset.csv")

#Ordenamos porque usaremos lags
data = data.sort_values(
    ["shop_id", "item_id", "date_block_num"]
).reset_index(drop=True)

(10913804, 6)


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id_x,item_category_id_y
0,1,0,12,0.0,55,55
1,0,0,19,0.0,40,40
2,0,0,27,0.0,19,19
3,1,0,27,0.0,19,19
4,0,0,28,0.0,30,30


In [3]:
for lag in [1, 2, 3]:
    data[f"lag_{lag}"] = (
        data
        .groupby(["shop_id", "item_id"])["item_cnt_month"] #Separamos las series de cada combinación tienda-item
        .shift(lag) #por shit() es que ordenamos antes la data, ya que shit() asume que las series están en el orden correcto
    )


In [4]:
#Convertimos todas las variables a promedios mensuales y les aplicamos el lag t-1
#Promedio histórico mensual del item
data["item_avg"] = (
    data
    .groupby("item_id")["item_cnt_month"]
    .transform("mean")
)


In [6]:
#Promedio histórico del item con lag t - 1
data["item_avg_lag_1"] = (
    data
    .groupby("item_id")["item_avg"]
    .shift(1)
)


In [10]:
#Promedio histórico mensual de ventas por tienda
shop_avg = (
    data
    .groupby(["date_block_num", "shop_id"], as_index=False)["item_cnt_month"]
    .mean()
    .rename(columns={"item_cnt_month": "shop_month_avg"})
)

data = data.merge(shop_avg, on=["date_block_num", "shop_id"], how="left")


In [11]:
#Promedio histórico mensual de ventas por tienda lag t-1
data["shop_month_avg_lag_1"] = (
    data
    .groupby("shop_id")["shop_month_avg"]
    .shift(1)
)


In [16]:
#Promedio histórico mensual por categoría
cat_avg = (
    data
    .groupby(["date_block_num", "item_category_id_x"], as_index=False)["item_cnt_month"]
    .mean()
    .rename(columns={"item_cnt_month": "cat_month_avg"})
)

data = data.merge(cat_avg, on=["date_block_num", "item_category_id_x"], how="left")


In [18]:
#Promedio histórico mensual por categoría t-1
data["cat_month_avg_lag_1"] = (
    data
    .groupby("item_category_id_x")["cat_month_avg"]
    .shift(1)
)


In [19]:
#Creamos un contador 0-11 para los meses, con esto vemos la relación que tienen los meses entre si (estacionalidad)

data["month_of_year"] = data["date_block_num"] % 12


(7971687, 16)


In [21]:
#Definimos variable dependiente e independientes

FEATURES = [
    "lag_1", "lag_2", "lag_3",
    "item_avg_lag_1",
    "shop_month_avg_lag_1",
    "cat_month_avg_lag_1",
    "month_of_year"
]

TARGET = "item_cnt_month"


In [23]:
data.to_csv("../data/new/feature_dataset.csv", index=False)
print("guardado")


guardado
